# Objektno programiranje

Objektno programiranje je stil programiranja (tako kot npr. funkcijsko programiranje), v katerem operiramo z razredi oz. njihovimi instancami - objekti. Objekte opisujejo atributi, spreminjamo pa jih lahko preko metod. Ta stil je pogosto v uporabi pri modeliranju resničnega sveta (npr. poslovnih procesov), saj lahko z objekti v programu predstavimo resnične objekte.

Da se izognemo podvajanju kode v različnih razredih, ste že spoznali koncept **dedovanja**, kjer izpeljani razred podeduje atribute in metode. Z dedovanjem povezan pojem je tudi **polimorfizem** (večobličnost), ki se nanaša na različne oblike enakih metod v izpeljanih razredih. V širšem pomenu je polimorfizem prisoten povsod v Pythonu, ker lahko kličemo iste funkcije z različnimi tipi in številom argumentov. Najprej si oglejmo še **enkapsulacijo** in **abstrakcijo** v Pythonu.

## Enkapsulacija

Pri enkapsulaciji gre za skrivanje podatkov (atributov, metod) v razredih. Tipično poznamo tri nivoje dostopa: *javni* (public), *zaščiteni* (protected) in *zasebni* (private). V Pythonu obstajajo ti nivoji zgolj kot dogovor. Javni so dostopni vsem. Zaščiteni so namenjeni razvijalcem, ki želijo npr. izpeljati nov razred. Zasebni so namenjeni samo interni rabi za delovanje razreda. V jezikih kot sta Java in C++ so ti dostopi strožje nadzorovani. Enaki dogovori veljajo tako za atribute kot za metode. Zaščiteni atributi se začnejo z enim podčrtajem, zasebni pa z dvema. Kljub temu lahko dostopamo do zasebnega atributa preko imena `_ImeRazreda__ImeAtributa`.

In [1]:
class Oseba:
    def __init__(self):
        self.ime = "Ana"  # public
        self._email = "ana@email.si"  # protected
        self.__vpisna = 12345678  # private
    def izpis(self):
        print(self.ime, self._email, self.__vpisna)
    def _preimenuj(self, ime):
        self.ime = ime
        self._email = f"{ime}@email.si"
    def __preveri(self):
        return len(str(self.__vpisna)) == 8  # znotraj razreda lahko dostopamo do zasebnih atributov

student = Oseba()
student.izpis()
#print(student.ime, student._email, student.__vpisna)  # neposreden dostop do zasebnih atributov od zunaj ni mogoč
print(student.ime, student._email, student._Oseba__vpisna)  # skrivanje imena (name mangling)

student._preimenuj("Andrej")
student.izpis()
print(student._Oseba__preveri())

Ana ana@email.si 12345678
Ana ana@email.si 12345678
Andrej Andrej@email.si 12345678
True


## Abstrakcija

Včasih želimo definirati zahteve, ki jim morajo zadoščati razredi. Na primer vsi razredi za izris morajo implementirati metodi draw in size, ne glede na to, ali je to razred za izris pravokotnika ali kroga. Takim razredom rečemo vmesniki (interface) ali abstraktni razredi. Predstavljajo vzorec oz. zahteve za izpeljane razrede, ne moremo pa ustvariti objekta takega abstraktnega razreda. V Pythonu morajo abstraktni razredi dedovati od razreda `ABC`, abstraktne metode pa označimo z dekoratorjem `abstractmethod`.

In [2]:
from abc import ABC, abstractmethod

class Zival(ABC):
    def __init__(self, ime):
        self.ime = ime

    @abstractmethod
    def glas(self):
        pass

#floki = Zival("Floki")  # Can't instantiate abstract class Zival with abstract methods glas

class Krava(Zival):
    def glas(self):
        print(f"{self.ime}: muuuu")

liska = Krava("Liska")
liska.glas()

Liska: muuuu


## Statične metode, atributi razreda

Metode, ki so neodvisne od konkretnega objekta (`self`) oz. njegovih atributov, imenujemo statične metode. Z uporabo dekoratorja `staticmethod` lahko do njih dostopamo preko razreda ali preko objekta. Atributi razreda so skupni vsem objektom tega razreda. Tipično so to kakšne konstante, ki so neodvisne od objekta. Na primer v razredu Kvadrat bi lahko imeli atribut stevilo_oglisc, ki je neodvisno od velikosti kvadrata, kar bi bil lahko atribut objekta. V spodnjem primeru pa izkoristimo deljenje atributa razreda med objekti in zgradimo množico vseh objektov, ki so bili ustvarjeni.

In [3]:
class Pes:
    @staticmethod
    def glas():  # statična metoda
        return "Hov"

    imena = set()  # atribut razreda

    def __init__(self, ime):
        self.ime = ime
        Pes.imena.add(ime)

fido = Pes("Fido")
floki = Pes("Floki")
print(Pes.glas(), fido.glas())
print(Pes.imena)

Hov Hov
{'Floki', 'Fido'}


## Posebne metode

Številne vgrajene funkcije se zanašajo na posebne metode, ki jih morajo implementirati razredi za pravilno delovanje. To so metode, ki se začnejo in končajo z dvema podčrtajema (poznamo že `__init__`, ki ima vlogo konstruktora). Tipično jih ne kličemo, ampak jih uporabljajo druge Pythonove funkcije. Na primer funkcija `len(seznam)` se izvede tako, da preloži delo na metodo `__len__` objekta `seznam`. Vrne torej rezultat klica `seznam.__len__()`. Podobno velja za cel kup drugih metod, operatorje primerjanja, aritmetične operacije, indeksiranje, ... Celoten seznam najdete v dokumentaciji https://docs.python.org/3/reference/datamodel.html#special-method-names.

In [4]:
s,t = [1,4,2], [1,5,0]
print("len:", len(s), s.__len__())
print("str:", str(s), s.__str__())
print("<  :", s < t, s.__lt__(t))

len: 3 3
str: [1, 4, 2] [1, 4, 2]
<  : True True


Implementacijo posebnih metod si bomo ogledali na lastnem razredu `Vector` za delo z vektorji v večdimenzionalnem prostoru. Konstruktor bo sprejel spremenljivo število argumentov, ki predstavljajo koordinate v različnih dimenzijah. Dolžina oz. velikost vektorja (`len`) naj bo kar število dimenzij. Metodi `str` in `repr` pa se uporabljata za predstavitev objekta v obliki niza. Prva je namenjena predstavitvi uporabnikom, druga pa razvijalcem. Pri izpisu funkcija `print` implicitno pretvori argumente s funkcijo `str`.

In [5]:
class Vector:
    def __init__(self, *args):
        self.k = list(args)
    def __len__(self):
        return len(self.k)
    # predstavitev
    def __str__(self):
        return f"Vector{self.k}"
    def __repr__(self):
        return repr(self.k)
    
    
v = Vector(3,-7,1)
print("len:", len(v))
print("str:", str(v), v)
print("repr:", repr(v))

len: 3
str: Vector[3, -7, 1] Vector[3, -7, 1]
repr: [3, -7, 1]


Dodajmo še indeksiranje oz. dostop do posameznih koordinat vektorja, kar storimo z metodama `__getitem__` in `__setitem__`.

In [6]:
class Vector:
    def __init__(self, *args):
        self.k = list(args)
    def __str__(self):
        return f"Vector{self.k}"
    # indeksiranje
    def __getitem__(self, item):
        return self.k[item]
    def __setitem__(self, key, value):
        self.k[key] = value

v = Vector(3,-7,1)
print(v[1])
v[2] = 9
print(v)

-7
Vector[3, -7, 9]


Dodajmo še podporo za seštevanje vektorjev z metodo `__add__`. Operacija `a+b` se namreč izvede kot `a.__add__(b)`. Pri tem moramo biti pozorni, da metoda vrne objekt tipa `Vector`, da lahko z njim izvajamo nadaljnje operacije. Množenje dveh vektorjev (`__mul__`) naj vrne skalarni produkt, množenje vektorja s številom pa raztegnjen vektor. Paziti moramo tudi na vrstni red operandov. Izraz `3*u` se namreč izračuna tako, da se na številu tipa `int` pokliče metoda `__mul__` z argumentom `u` tipa `Vector`. Ker `int` sploh ne ve, da `Vector` obstaja, operacija ne uspe. Namesto tega Python poskusi izvesti metodo za množenje z desne strani `__rmul__`. To metodo smo implementirali vektorju, da se zna pomnožiti s številom, tudi če je vektor desni operand.

In [7]:
class Vector:
    def __init__(self, *args):
        self.k = list(args)
    def __str__(self):
        return f"Vector{self.k}"
    # aritmetika
    def __add__(self, other):
        return Vector(*[x+y for x,y in zip(self.k, other.k)])
    def __mul__(self, other):
        if isinstance(other, Vector):
            return sum(x*y for x, y in zip(self.k, other.k))
        else:
            return Vector(*[x*other for x in self.k])
    def __rmul__(self, other):
        return self*other

u = Vector(-1,2,3)
v = Vector(3,-7,1)
print(u+v+u)
print(u*v)
print(u*3)
print(3*u)  # __rmul__

Vector[1, -3, 7]
-14
Vector[-3, 6, 9]
Vector[-3, 6, 9]


Vektorje bomo primerjali med seboj glede na dolžino vektorja (Evklidska norma). Definirali bomo operator enakosti (`__eq__`) in operator manjši (`__lt__`). Neenakost (`!=`) je negacija enakosti, če ni drugače definirano z metodo `__ne__`. Preostale operatorje pa bo v tem primeru namesto nas definiral dekorator `functools.total_ordering` (https://docs.python.org/3/library/functools.html#functools.total_ordering).

In [8]:
from math import sqrt
from functools import total_ordering

@total_ordering
class Vector:
    def __init__(self, *args):
        self.k = list(args)
    def __len__(self):
        return len(self.k)
    def __str__(self):
        return f"Vector{self.k}"
    def norm(self):
        return sqrt(sum(x**2 for x in self.k))
    # primerjanje
    def __lt__(self, other):
        return self.norm() < other.norm()
    def __eq__(self, other):
        return self.norm() == other.norm()


u = Vector(-1,2,3)
v = Vector(3,-7,1)
print("manjse:", u < v)
print("(ne)enakost:", u==v, u!=v)
print("ostalo:", u<=v, u>=v, u>v)

manjse: True
(ne)enakost: False True
ostalo: True False False


Tudi klic objekta ni nič drugega kot klic posebne metode `__call__`. Vektor lahko interpretiramo kot funkcijo, ki izvede premik argumenta. Argument je v tem primeru točka, ki jo lahko predstavimo z vektorjem. Klic vektorja z vektorjem kot argumentom naj bo torej enak seštevanju.

In [9]:
class Vector:
    def __init__(self, *args):
        self.k = list(args)
    def __str__(self):
        return f"Vector{self.k}"
    def __add__(self, other):
        return Vector(*[x+y for x,y in zip(self.k, other.k)])
    # klic
    def __call__(self, other):
        return self+other


tocka = Vector(-1,2,3)
premik = Vector(3,-7,1)
nova = premik(tocka)
print(nova)

Vector[2, -5, 4]


## Večkratno, večstopenjsko dedovanje

Večstopenjsko dedovanje že poznate. Gre zgolj za več nivojev dedovanja, kjer je iz razreda A izpeljan razred B, iz njega razred C, iz C-ja D, itd. Stvari pa se zakomplicirajo pri večkratnem dedovanju (*multiple inheritance*), kjer razred deduje od dveh ali več razredov. Glede na hierarhijo dedovanja razredov, se lahko srečamo z različnimi problemi, kar si bomo ogledali na spodnjem primeru razredov, ki opisujejo like.

Hierarhija likov

Zanimiv razred je `Kvadrat`, ki je izpeljan iz razredov `Stirikotnik` in `Pravilen`. Do metod in atributov dedovanega razreda smo navajeni dostopati z uporabo funkcije `super()`. V tem primeru pa imamo dva razreda, od katerih dedujemo metode. Do metod dostopamo preko imena razreda, objekt `self` pa moramo lastnoročno posredovati klicani metodi.

In [10]:
class Lik:
    def tip(self): return "Lik"
class Krog(Lik):
    def tip(self): return "Krog"
class Veckotnik(Lik):
    def tip(self): return "Veckotnik"
class Pravilen(Veckotnik):
    def tip(self): return "Pravilen"
class Stirikotnik(Veckotnik): pass
class Kvadrat(Stirikotnik, Pravilen):
    def tip(self): 
        return Stirikotnik.tip(self) + " & " + Pravilen.tip(self)  # dostop do metod različnih dedovanih razredov

print(Kvadrat().tip())

Veckotnik & Pravilen


Sedaj si oglejmo situacijo, ko sta razreda `Stirikotnik` in `Kvadrat` brez metode `tip`. Katera metoda se izvede pri klicu metode `tip` v razredu `Kvadrat`, metoda razreda `Pravilen` ali `Veckotnik`?

In [11]:
class Stirikotnik(Veckotnik): pass
class Kvadrat(Stirikotnik, Pravilen): pass

print(Kvadrat().tip())

Pravilen


Vrstni red iskanja manjkajočih atributov in metod definira *Method Resolution Order*. Zanj velja, da se izpeljani razredi (nižje v hierarhiji) pojavijo pred dedovanimi razredi (višje v hierarhiji). Hkrati pa ohranja vrstni red razredov pri večkratnem dedovanju. Če razred deduje od več drugih razredov, se bodo ti dedovani razredi pojavili v takem vrstnem redu, kot so našteti pri dedovanju. Kogar zanima več o MRO, si lahko ogleda https://www.python.org/download/releases/2.3/mro/.

In [12]:
Kvadrat.mro()

[__main__.Kvadrat,
 __main__.Stirikotnik,
 __main__.Pravilen,
 __main__.Veckotnik,
 __main__.Lik,
 object]

Na naslednjo težavo naletimo pri konstruktorjih. Bi moral konstruktor kvadrata klicati konstruktorja štirikotnika in pravilnega? V tem primeru bi vsak od njiju poklical konstruktor večkotnika, torej bi ga izvedli dvakrat, česar pa nočemo.

Kljub večkratnemu dedovanju sledimo vzorcu uporabe funkcije `super()`, ki smo ga navajeni. Ta poskrbi, da se izvedejo konstruktorji potrebnih razredov v pravem vrstnem redu in vsak enkrat (https://docs.python.org/3/library/functions.html#super), pri tem pa si pomaga z MRO.

Napišimo dekorator `log`, ki bo izpisal, kdaj se določena metoda začne in zaključi. Z njim dekorirajmo vse konstruktorje in opazujmo vrstni red klicev. Zaporedje klicev je `Kvadrat`, `Stirikotnik`, `Pravilen`, ... Funkcija `super()` v razredu `Stirikotnik` torej pokliče `__init__` v razredu `Pravilen`, čeprav ne dedujeta med seboj!

In [13]:
def log(f):
    def wrapper(*args, **kwargs):
        print(f.__qualname__ , "start")
        f(*args, **kwargs)
        print(f.__qualname__ , "end")
    return wrapper

class Lik:
    @log
    def __init__(self): super().__init__()
class Krog(Lik):
    @log
    def __init__(self): super().__init__()
class Veckotnik(Lik):
    @log
    def __init__(self): super().__init__()
class Pravilen(Veckotnik):
    @log
    def __init__(self): super().__init__()
class Stirikotnik(Veckotnik):
    @log
    def __init__(self): super().__init__()
class Kvadrat(Stirikotnik, Pravilen):
    @log
    def __init__(self): super().__init__()

k = Kvadrat()  # super() iz razreda Stirikotnik pokliče __init__ iz razreda Pravilen!

Kvadrat.__init__ start
Stirikotnik.__init__ start
Pravilen.__init__ start
Veckotnik.__init__ start
Lik.__init__ start
Lik.__init__ end
Veckotnik.__init__ end
Pravilen.__init__ end
Stirikotnik.__init__ end
Kvadrat.__init__ end


## Iteratorji

Številni razredi oz. objekti v Pythonu omogočajo iteracijo — po domače sprehajanje čez objekt. Na primer v seznamih in množicah se lahko s for zanko sprehodimo čez vse elemente, v slovarjih pa čez vse ključe.

In [14]:
sez = [1,2,3]
for x in sez:
    print(x)

1
2
3


Oglejmo si, kako to pravzaprav deluje. S funkcijo `iter` se iz objekta ustvari iterator. Ta iterator lahko uporabljamo v kombinaciji s funkcijo `next` za generiranje elementov. Ko zmanjka elementov za iteracijo, sproži `next` izjemo `StopIteration`. Spodnja koda prikazuje delovanje for zanke.

In [15]:
it = iter(sez)
while True:
    try:
        x = next(it)
        print(x)
    except StopIteration: break

1
2
3


Ustvarimo lasten razred, ki bo podpiral iteracijo. To bomo naredili na primeru razreda učencev, ki morajo pred tablo reševati naloge. Konstruktor razreda bo sprejel seznam učencev in števil nalog, ki jih je treba rešiti. Z iteracijo čez razred pa želimo dobiti naključen izbor toliko učencev, kot je nalog. Da bomo lahko iterirali čez razred, mora ta podpirati funkcijo `iter` z implementacijo metode `__iter__`. Vrniti mora iterator oz. objekt, ki bo imel implementirano metodo `__next__`, ki jo potrebuje funkcija `next`. To je lahko tudi isti objekt (`self`).

In [16]:
from random import choice

class Razred:
    def __init__(self, ucenci, naloge):
        self.ucenci = ucenci
        self.naloge = naloge
    def __iter__(self):
        return self
    def __next__(self):
        if self.naloge == 0: raise StopIteration
        self.naloge -= 1
        return choice(self.ucenci)

r = Razred(["Ana", "Miha", "Tone", "Metka", "Janez", "Beti"], 4)
print([oseba for oseba in r])

print([oseba for oseba in r])  # ne deluje

['Tone', 'Janez', 'Tone', 'Tone']
[]


Enkratna iteracija čez naš razred deluje, druga pa ne več. Težava je v tem, da število nalog med prvo iteracijo pade na 0 in tam tudi ostane, zato se druga iteracija takoj zaključi. Popravimo napako z novo pomožno spremenljivko.

In [17]:
class Razred:
    def __init__(self, ucenci, naloge):
        self.ucenci = ucenci
        self.naloge = naloge
    def __iter__(self):
        self.n = self.naloge
        return self
    def __next__(self):
        if self.n == 0: raise StopIteration
        self.n -= 1
        return choice(self.ucenci)

r = Razred(["Ana", "Miha", "Tone", "Metka", "Janez", "Beti"], 4)
print([oseba for oseba in r])
print([oseba for oseba in r])

print([[(oseba1, oseba2) for oseba2 in r] for oseba1 in r])  # ne deluje

['Metka', 'Miha', 'Beti', 'Beti']
['Miha', 'Miha', 'Metka', 'Janez']
[[('Janez', 'Ana'), ('Janez', 'Metka'), ('Janez', 'Metka'), ('Janez', 'Janez')]]


Sedaj imamo težavo z gnezdenimi zankami, če iteriramo čez isti objekt. Odvisno od namena uporabe razreda, je to lahko problem, ali pa tudi ne. Rešimo ga lahko tako, da metoda `__iter__` vrne nov objekt - naključni iterator `RandIter`, ki poskrbi za iteracijo. V primerjavi s prejšnjo rešitvijo se ob vsakem začetku iteracije ustvari nov iterator.

In [18]:
class Razred:
    def __init__(self, ucenci, naloge):
        self.ucenci = ucenci
        self.naloge = naloge
    def __iter__(self):
        return RandIter(self.ucenci, self.naloge)

class RandIter:
    def __init__(self, sez, n):
        self.sez = sez
        self.n = n
    def __next__(self):
        if self.n == 0: raise StopIteration
        self.n -= 1
        return choice(self.sez)

r = Razred(["Ana", "Miha", "Tone", "Metka", "Janez", "Beti"], 4)
print([oseba for oseba in r])
print([oseba for oseba in r])
print([[(oseba1, oseba2) for oseba2 in r] for oseba1 in r])

['Janez', 'Janez', 'Metka', 'Miha']
['Metka', 'Beti', 'Ana', 'Ana']
[[('Beti', 'Metka'), ('Beti', 'Tone'), ('Beti', 'Ana'), ('Beti', 'Metka')], [('Beti', 'Beti'), ('Beti', 'Ana'), ('Beti', 'Beti'), ('Beti', 'Miha')], [('Janez', 'Metka'), ('Janez', 'Ana'), ('Janez', 'Ana'), ('Janez', 'Janez')], [('Janez', 'Beti'), ('Janez', 'Miha'), ('Janez', 'Metka'), ('Janez', 'Tone')]]
